# Introduction to Gravitational Lensing

FIXME redo with physical deflection angle, reduced deflection angle, and dimensionless deflection angle???

Here we provide a primer course on gravitational lensing. It includes interactive examples using `caustics` to demo various effects.

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle
from IPython.display import HTML
import numpy as np

import caustics

In [ ]:
# setup stuff to define a grid of points to make images

n_pix = 256
res = 0.015
fov = n_pix * res
upsample_factor = 2
z_l = torch.tensor(0.5, dtype=torch.float32)
z_s = torch.tensor(1.0, dtype=torch.float32)

## Lensing Formalism

### Configuration of a Lens

Before we get too deep into the weeds, lets set the stage. A photon has arrived in your telescope and you are wondering where it came from. Well if it encountered some mass on its path to you (and really what photon hasn't?) then the path it took might look something like this:

![Lens configuration](assets/lensconfiguration.png)

*Figure taken from Bartelmann and Schneider 2001 (figure 11)*

The center of your image is your (0,0) coordinate and this corresponds to the *optical axis* which is the dashed line. The source of the photon is said to exist in the *source plane* which is a distance of $D_s$ from the observer. The mass that deflected the photon is said to exist in the *lens plane* which is a distance $D_l$ (I use $D_l$ rather that $D_d$ for the sake of my sanity) from the observer and $D_{ls}$ from the source plane. Finally, you receive the photons in the *image plane* at the observer. Note that cosmological distances are so large that these may be very accurately treated as infinitely thin planes, even if the source and the mass are in fact 3D structures. We'll talk a little about extended mass distributions in the [Multiplane Lensing](#multiplane-lensing) section. In the source plane the photons are coming from a point $\vec{\eta}$ away from the optical axis, this is a physical distance you could measure in kilometers if you like. In the lens plane, the photon intersected at $\vec{\xi}$ away from the optical axis (again physical units). In the image plane we can say the photon was essentially at the optical axis, though it was probably a few cm/meters away from it in reality when it struck the telescope.

As an observer, you measure everything in angular units. The photon is the solid line which you see at some position $\vec{\theta}$ off from the optical axis. Had there been no mass intercepting the photon, it would have never reached your telescope, instead a photon coming from the same source would have arrived at $\vec{\beta}$ angular units from your optical axis. The correction at the source plane which shows how much the photon deviated from its original path is called the *deflection angle* $\hat{\vec{\alpha}}$ and it is essentially a measure of the impact lensing has had on your observation. Note that in the figure the deflection angle is drawn as if the photon traveled backwards, leaving your telescope at angle $\vec{\theta}$ getting to the lens plane, getting deflected by $\hat{\vec{\alpha}}$, and finally arriving at the source plane. This is on purpose since the geometry of lensing works in either direction, but computationally it is easier to go backwards (we'll see why in [the lens equation section](#the-lens-equation)).

These are the names and symbols used to describe the configuration of most lensing systems. So far we haven't told you how any of this came to be, but hopefully it will give you a reference point to understand what comes next.

### The Lens Equation

With some simple trigonometry it is possible to relate the physical sizes and the angular sizes:

$$\vec{\eta} = \vec{\beta} D_s$$

$$\vec{\xi} = \vec{\theta} D_l$$

Just note that the distances must be in angular diameter distances for these relations to hold (essentially by definition). We can also go a step further and get $\vec{\eta}$ using $\vec{\theta}$ and $\hat{\vec{\alpha}}$:

$$\vec{\eta} = \vec{\theta} D_s - \hat{\vec{\alpha}} D_{ls}$$

From here we get the lens equation using the two $\vec{\eta}$ formulas:

$$\vec{\beta} D_s = \vec{\theta} D_s - \hat{\vec{\alpha}} D_{ls}$$

We are tantalizingly close to just cancelling those $D_s$ distances. How about we make a definition $\vec{\alpha} \equiv \frac{D_{ls}}{D_s}\hat{\vec{\alpha}}$ then we can write the lens equation as:

$$\vec{\beta} = \vec{\theta} - \vec{\alpha}$$

Excellent! Just note that we had to change $\hat{\vec{\alpha}}$ which is the real angular deflection angle that we can do geometry with, to now be $\vec{\alpha}$ which has a convenient refactoring such that the lens equation is easier to work with. The modified $\vec{\alpha}$ is called the *reduced deflection angle* and usually other quantities that have had a similar transformation to make the lens equation come out nice are called reduced quantities.

This is a very nice and simple equation, one might be fooled into thinking it is easy to work with. Now lets explain why one typically works backwards with photons "coming from" the observer and going to the source plane. The deflection angle for almost all lenses is a function of the position in the sky (more deflection closer to the mass, less deflection further away) meaning that it is in fact a function $\vec{\alpha}(\vec{\theta})$. So to compute $\vec{\beta}$ when you know $\vec{\theta}$ means simply computing $\vec{\alpha}(\vec{\theta})$ and the taking the difference $\vec{\theta} - \vec{\alpha}(\vec{\theta})$. But if you want to compute $\vec{\theta}$ only knowing $\vec{\beta}$, then you will need to search around for a value to input into $\vec{\alpha}(\vec{\theta})$ until you can get the equation to be satisfied, which means you need to know $\vec{\theta}$ to compute $\vec{\theta}$! This is a source of great consternation in gravitational lensing researchers, but is a fact we must deal with. In the next sections we'll get into more specifics on all the interesting results that come from this simple equation.

In [ ]:
F = 50

# Define a cosmology for the lensing
cosmology = caustics.FlatLambdaCDM()
# Define a point mass for the lens plane
lens = caustics.Point(cosmology=cosmology, x0=0.0, y0=0.0, th_ein=1.0, z_l=z_l)

# Make a bunch of theta values at which to raytrace
theta_x = torch.linspace(-fov / 2, fov / 2, F, dtype=torch.float32)
theta_y = torch.zeros(F, dtype=torch.float32)
# Evaluate the reduced deflection angles
alpha_x, alpha_y = lens.reduced_deflection_angle(theta_x, theta_y, z_s)
# Compute the lens equation at these coordinates
beta_x, beta_y = theta_x - alpha_x, theta_y - alpha_y

This gives a nice dynamic view of what happens to light rays as they are raytraced through a gravitational lens system. Here we show two perspectives, one is from the angular coordinates where one typically works to generate images, the other is the physical raytracing of a ray in real coordinates (redshift on x-axis, Mpc on y-axis). 

In [ ]:
# Create animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))

Dl = cosmology.angular_diameter_distance(z_l)
Ds = cosmology.angular_diameter_distance(z_s)


def update(frame):
    """Update function for the animation."""
    # Update the image in the first subplot
    ax1.clear()
    ax1.scatter([z_l.item()], [0], color="b", label="Point mass")
    ax1.plot(
        [0, z_l.item()],
        [theta_x[frame].item(), theta_x[frame].item()],
        color="r",
        label="Light Ray",
    )
    ax1.plot(
        [z_l.item(), z_s.item()],
        [beta_x[frame].item(), beta_x[frame].item()],
        color="r",
    )
    ax1.plot(
        [z_l.item(), z_l.item()],
        [theta_x[frame].item(), beta_x[frame].item()],
        color="r",
    )
    ax1.text(
        z_l.item() * 0.5, theta_x[frame].item(), "$\\theta$", fontsize=16, color="k"
    )
    ax1.text(
        (z_l.item() + z_s.item()) * 0.5,
        beta_x[frame].item(),
        "$\\beta$",
        fontsize=16,
        color="k",
    )
    ax1.text(
        z_l.item(),
        (theta_x[frame] + beta_x[frame]).item() * 0.5,
        "$\\alpha$",
        fontsize=16,
        color="k",
    )
    ax1.axvline(
        x=z_l.item(), color="grey", linestyle="--", linewidth=2, label="Lens Plane"
    )
    ax1.axvline(
        x=z_s.item(), color="grey", linestyle="-", linewidth=2, label="Source Plane"
    )
    ax1.axvline(x=0, color="grey", linestyle=":", linewidth=2, label="Image Plane")
    ax1.legend(loc="upper left")
    ax1.set_ylim(-fov / 1.9, fov / 1.9)
    ax1.set_title("Raytracing (Angular Coordinates)")
    ax1.set_xlabel("redshift z")
    ax1.set_ylabel("Light ray angle [arcsec]")

    ax2.clear()
    ax2.scatter([z_l.item()], [0], color="b", label="Point mass")
    ax2.plot(
        [0, z_l.item()],
        [0.0, (Dl * theta_x[frame]).item()],
        color="r",
        label="Light Ray",
    )
    ax2.plot(
        [z_l.item(), z_s.item()],
        [(Dl * theta_x[frame]).item(), (Ds * beta_x[frame]).item()],
        color="r",
    )
    ax2.axvline(
        x=z_l.item(), color="grey", linestyle="--", linewidth=2, label="Lens Plane"
    )
    ax2.axvline(
        x=z_s.item(), color="grey", linestyle="-", linewidth=2, label="Source Plane"
    )
    ax2.text(z_l.item(), (Dl * theta_x[frame]).item(), "$\\xi$", fontsize=16, color="k")
    ax2.text(z_s.item(), (Ds * beta_x[frame]).item(), "$\\eta$", fontsize=16, color="k")
    ax2.scatter([0], [0], color="grey", label="Image Plane")
    # ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.set_ylim(-abs(Ds * beta_x[0].item()) * 1.1, abs(Ds * beta_x[0].item()) * 1.1)
    ax2.set_title("Raytracing (Physical Coordinates)")
    ax2.set_xlabel("redshift z")
    ax2.set_ylabel("Physical distance [Mpc]")

    return


update(0)

ani = animation.FuncAnimation(fig, update, frames=F, interval=80)

plt.close()
HTML(ani.to_jshtml())

### Lens Convergence

Lensing is sensitive to all mass on the line of sight. Typically, we describe astronomical massive objects via a 3D density $\rho(\vec{\theta},z)$, but in gravitational lensing we may describe objects as existing on an infinitely thin lens plane. The relevant quantity is then the *surface density* which we may determine by integrating over $z$ to get:

$$\Sigma(\vec{\theta}) = \int\rho(\vec{\theta}, z)dz$$

This is often normalized by the *critical surface density* at the lens plane $\Sigma_{cr} = \frac{c^2D_s}{4\pi G D_lD_{ls}}$. This is different from the cosmological critical density $\rho_c = \frac{3H^2}{8\pi G}$, where $H$ is the expansion rate at a given redshift (i.e. $H_0\approx 70 \frac{\rm km}{{\rm s}\cdot {\rm Mpc}}$ is the expansion rate today). This allows us to compute a dimensionless surface density or *convergence*:

$$\kappa(\vec{\theta}) = \frac{\Sigma(\vec{\theta})}{\Sigma_{cr}}$$

The convergence is very useful as it is a dimensionless way to describe the cumulative mass distribution along the line of sight (at least within our lens plane, we'll get more into this in the [multiplane section](#multiplane-lensing)). For some intuition on the convergence, note that a convergence at or above 1 indicates that strong lensing (multiple images) will occur. It is also the most easily translated quantity to say numerical simulations or physical models since we can simply take a projected mass histogram to get the convergence.

Below we have plotted some convergence maps for various lensing mass distributions. In the second subfigure we included a convergence map computed from a numerical simulation (Illustris TNG) by essentially taking a 2D histogram of particle masses.

In [ ]:
cosmology = caustics.FlatLambdaCDM()
theta_x, theta_y = caustics.utils.meshgrid(res, n_pix)

fig, axarr = plt.subplots(1, 5, figsize=(25, 5))
plt.subplots_adjust(wspace=0.01)
fig.suptitle("Lens Convergence Examples")
kappa_map = np.load("tutorials/assets/kappa_maps.npz")["kappa_maps"][1]
lenses = [
    caustics.SIE(cosmology=cosmology, x0=0, y0=0, q=0.6, phi=np.pi / 3, b=1, z_l=z_l),
    caustics.PixelatedConvergence(
        cosmology=cosmology,
        x0=0,
        y0=0,
        convergence_map=kappa_map,
        pixelscale=fov / kappa_map.shape[0],
        z_l=z_l,
    ),
    caustics.NFW(cosmology=cosmology, x0=0, y0=0, m=1e12, c=5, z_l=z_l),
    caustics.PseudoJaffe(
        cosmology=cosmology,
        x0=0,
        y0=0,
        mass=1e12,
        core_radius=0.2,
        scale_radius=1.0,
        z_l=z_l,
    ),
    caustics.Multipole(
        cosmology=cosmology, m=(3,), x0=0, y0=0, a_m=[2], phi_m=[np.pi / 3], z_l=z_l
    ),
]
for i, lens in enumerate(lenses):
    k = lens.convergence(theta_x, theta_y, z_s=z_s)
    k = k.log() if i < 4 else k.tanh()
    axarr[i].imshow(k.numpy())
    axarr[i].set_title(lens.__class__.__name__)
    axarr[i].axis("off")
plt.show()

### Lens Potential

There are many ways to describe a gravitational lens, with various advantages. Here we will describe the lensing potential $\Psi$, which is much like the gravitational potential except projected into the 2D plane of our infinitely thin lens:

$$\hat{\Psi}(\vec{\theta}) = \frac{2D_{ls}}{D_lD_sc^2}\int\phi(D_l\vec{\theta},z)dz$$

And the dimensionless version of this potential is:

$$\Psi(\vec{\theta}) = \frac{D_l^2}{\xi_0^2}\hat{\Psi}$$

A convenient property of the lensing potential is that it encodes all the lens properties locally

### Relating the deflection angle, convergence, and potential

The simplest to convert is the potential:

$$\vec{\alpha}(\vec{\theta}) = \vec{\nabla}_{\vec{\theta}}\Psi(\vec{\theta})$$

$$\kappa(\vec{\theta}) = \frac{1}{2}\triangle_{\vec{\theta}}\Psi(\vec{\theta})$$

Where $\triangle_{\vec{\theta}} = \frac{\partial^2}{\partial\theta_0^2} + \frac{\partial^2}{\partial\theta_1^2}$ is the Laplacian. The potential encodes the lensing information locally, what this means is that one may take derivatives to get the other quantities. There is no need to perform any integrals over all space with the potential. This is a useful property, especially in `caustics` where all functions are automatically differentiable.

Next we can see how the convergence transforms:

$$\Psi(\vec{\theta}) = \frac{1}{\pi}\int\kappa(\vec{\theta}')\ln|\vec{\theta} - \vec{\theta}'|d\vec{\theta}'$$

$$\vec{\alpha}(\vec{\theta}) = \frac{1}{\pi}\int\kappa(\vec{\theta}')\frac{\vec{\theta}-\vec{\theta}'}{|\vec{\theta}-\vec{\theta}'|}d\vec{\theta}'$$

Opposite to the potential, one must perform integrals over all space to convert the convergence into other quantities. One convenience is that these integrals are framed as convolutions, so one may use efficient numerical algorithms (Fast Fourier Transforms) to compute reasonable approximations of these integrals.

It is not common to convert from the deflection angle to either the potential or convergence as this would involve inverting the above equations. This cannot be done in general and so an iterative method would likely be needed.

## Deflection of a Point Mass

In this tutorial we will not be too concerned with the derivation of lensing quantities, instead focusing on intuition and core concepts (see [Fleury et al. 2022](https://iopscience.iop.org/article/10.1088/1361-6382/abea2d), the [Meneghetti lecture notes](https://www.ita.uni-heidelberg.de/~jmerten/misc/meneghetti_lensing.pdf), and [Bartelmann and Schneider 2001](https://doi.org/10.1016/S0370-1573(00)00082-X)). So now let's see the deflection angle for a point mass:

$$|\hat{\vec{\alpha}}| = \frac{4GM}{c^2|\vec{\theta}|}$$

We use $|\hat{\vec{\alpha}}|$ and $|\vec{\theta}|$ because angular positions on the sky are 2D vectors, but in this equation we only need to know the magnitudes. A point mass is symmetric about its center with the deflection always pointing inwards. For simplicity we have assumed the point mass is on the optical axis, you can of course translate the point mass to other parts of the field of view. Notice that the deflection drops off with distance from the point mass, it should not be too surprising that the amount of deflection decreases further from the mass.

The above deflection is the physical deflection angle which is in regular units. If we convert to reduced deflection angle then things become simpler (as usual).

$$|\vec{\alpha}| = \frac{R_e^2}{|\vec{\theta}|}$$

where $R_e$ is the Einstein radius, which is the angular radius at which the point mass has a critical curve with (technically) infinite magnification. We'll get to [magnification](#magnification-and-shear) in a later section. The Einstein radius is defined as:

$$R_e = \sqrt{\frac{4GMD_{ls}}{c^2D_lD_s}}$$

Let's see what it looks like for something to be lensed by a point mass. We will use a gaussian blob on the source plane and have it pass behind the point mass.

In [ ]:
# Define a gaussian blob for the source plane
source = caustics.Sersic(q=1.0, phi=0.0, n=0.5, Re=0.3, Ie=1.0)
# Define a cosmology for the lensing
cosmology = caustics.FlatLambdaCDM()
# Define a point mass for the lens plane
lens = caustics.Point(cosmology=cosmology, x0=0.0, y0=0.0, th_ein=1.0, z_l=z_l)

# Make a bunch of theta values at which to raytrace
theta_x, theta_y = caustics.utils.meshgrid(res, n_pix)
# Evaluate the reduced deflection angles
alpha_x, alpha_y = lens.reduced_deflection_angle(theta_x, theta_y, z_s)
# Compute the lens equation at these coordinates
beta_x, beta_y = theta_x - alpha_x, theta_y - alpha_y

# Evaluate the brightness with the source moving along the x-axis
source_x = torch.linspace(-2, 2, 100)
source_y = torch.zeros(100)
unlensed_images = torch.vmap(lambda x: source.brightness(theta_x, theta_y, x))(
    torch.stack((source_x, source_y), dim=1)
)
lensed_images = torch.vmap(lambda x: source.brightness(beta_x, beta_y, x))(
    torch.stack((source_x, source_y), dim=1)
)

Here we show the deflection angles as a function of position on the sky. The reduced deflection angle is a vector quantity, so it has two components, the x deflection and the y deflection. We plot them side by side here. You can see that the deflection is strongest near the point mass and weakest further out.  

In [ ]:
# Visualize the reduced deflection angle
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 4))
ax1.imshow(alpha_x, origin="lower", cmap="seismic")
ax1.set_title("Reduced deflection angle [$\\alpha_x$]")
ax1.axis("off")
im = ax2.imshow(alpha_y, origin="lower", cmap="seismic")
fig.colorbar(im, ax=ax3, label="deflection magnitude [arcsec]")
ax3.axis("off")
ax2.set_title("Reduced deflection angle [$\\alpha_y$]")
ax2.axis("off")
plt.show()

Now we show what it looks like for a light source passing behind a point mass lens. This is just a gaussian moving from one side of the screen to the other. We have added a point where the lens mass is, and a circle to show the Einstein radius. You will see that when the lensed image reaches the full distortion (the source is exactly aligned with the point mass) the ring it creates aligns exactly with the Einstein radius.

In [ ]:
# Create animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# Display the first frame of the image in the first subplot
im1 = ax1.imshow(
    unlensed_images[0].numpy(), cmap="grey", interpolation="bilinear", vmin=0, vmax=2
)
ax1.set_title("unlensed")
ax1.axis("off")
im2 = ax2.imshow(
    lensed_images[0].numpy(),
    cmap="grey",
    interpolation="bilinear",
    vmin=0,
    vmax=2,
    extent=[-fov / 2, fov / 2, -fov / 2, fov / 2],
)
ax2.scatter([0], [0], color="r")
c = Circle((0, 0), lens.th_ein.value.item(), fill=False, color="r", linestyle="--")
ax2.add_patch(c)
ax2.set_title("lensed")
ax2.axis("off")


def update(frame):
    """Update function for the animation."""
    # Update the image in the first subplot
    im1.set_array(unlensed_images[frame].numpy())
    im2.set_array(lensed_images[frame].numpy())

    return im1, im2


ani = animation.FuncAnimation(fig, update, frames=lensed_images.shape[0], interval=60)

plt.close()
HTML(ani.to_jshtml())

## Deflection of an Extended Mass

Some astronomical objects like black holes and stars are often represented as point masses, but many others such as galaxies and clusters are extended mass distributions. These may be thought of as many point masses combined:

$$\hat{\vec{\alpha}}(\vec{\theta}) = \sum_i\hat{\vec{\alpha}}_i(\vec{\theta}_i - \vec{\theta}) = \frac{4G}{c^2}\sum_iM_i\frac{\vec{\theta}_i - \vec{\theta}}{|\vec{\theta}_i - \vec{\theta}|^2}$$

where the $\vec{\theta}_i$ are the positions of the point masses. In the limit that the extended mass is a continuous distribution, the $M_i$ become a density distribution $\rho(\vec{\theta}, z)$ where $z$ is the dimension along the line of sight. Of course for lensing we will use the convergence $\kappa(\vec{\theta})$ since a 3D density is more than we need.

### Singular Isothermal Sphere (SIS)

There are many mass distributions defined by different surface densities. One of the simplest comes from assuming that the mass of the lens is in the form of an ideal gas in a spherical potential. In thermal and hydrostatic equilibrium the density is given as:

$$\rho(r) = \frac{\sigma_v^2}{2\pi Gr^2}$$

$$\Sigma(|\vec{\theta}|) = \frac{\sigma_v^2}{2G|\vec{\theta}|}$$ 

<!--fixme reduced/dimensionless??? -->

$$\kappa = \frac{1}{2|\vec{\theta}|}$$

And the deflection angle is given by:

$$\vec{\alpha}(\vec{\theta}) = \frac{\vec{\theta}}{|\vec{\theta}|}$$

### Elliptical lens Mass

For any spherically symmetric mass distribution, such as the SIS it is possible to determine a change of coordinates such that the mass has been compressed along one axis, making it an elliptical mass distribution. The *axis ratio* of the ellipse $q = \frac{b}{a}$ is the ratio of semi-minor to semi-major axis lengths and it defines the ellipse. This looks like:

$$r\to R = \sqrt{\frac{x_1^2}{q} + qx_2^2}$$

In this transformed coordinate space the deflection angles become:

$$\alpha_x = \frac{x_1}{qR}\tilde{\alpha}_x(R)$$

$$\alpha_y = \frac{qx_2}{R}\tilde{\alpha}_y(R)$$

where $\tilde{\alpha}$ is the unmodified deflection angle of the spherical mass distribution.

### Singular Isothermal Ellipsoid (SIE)

An SIE is just like an SIS except we've converted it into an elliptical mass distribution. Lets try a video like we had in the [point mass section](#deflection-of-a-point-mass). Now we will use an SIE with an axis ratio of 0.5 to see what happens to the background source.

In [ ]:
# Define a gaussian blob for the source plane
source = caustics.Sersic(q=1.0, phi=0.0, n=0.5, Re=0.05, Ie=1.0)
# Define a cosmology for the lensing
cosmology = caustics.FlatLambdaCDM()
# Define a point mass for the lens plane
lens = caustics.SIE(cosmology=cosmology, x0=0.0, y0=0.0, q=0.5, phi=0, b=1.0, z_l=z_l)
# Define a caustics simulator to handle the raytracing
sim = caustics.LensSource(lens, source, pixelscale=res, pixels_x=n_pix, z_s=z_s)

# Evaluate the brightness with the source moving along the x-axis
source_x = torch.linspace(-2, 2, 100)
source_y = torch.zeros(100) + 0.1
unlensed_images = torch.vmap(lambda x: sim(x, lens_source=False))(
    torch.stack((source_x, source_y), dim=1)
)
lensed_images = torch.vmap(sim)(torch.stack((source_x, source_y), dim=1))

In [ ]:
# Create animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# Display the first frame of the image in the first subplot
im1 = ax1.imshow(
    unlensed_images[0].numpy(), cmap="grey", interpolation="bilinear", vmin=0, vmax=2
)
ax1.set_title("unlensed")
ax1.axis("off")
im2 = ax2.imshow(
    lensed_images[0].numpy(),
    cmap="grey",
    interpolation="bilinear",
    vmin=0,
    vmax=2,
    extent=[-2, 2, -2, 2],
)
ax2.scatter([0], [0], color="r")
ax2.set_title("lensed")
ax2.axis("off")


def update(frame):
    """Update function for the animation."""
    # Update the image in the first subplot
    im1.set_array(unlensed_images[frame].numpy())
    im2.set_array(lensed_images[frame].numpy())

    return im1, im2


ani = animation.FuncAnimation(fig, update, frames=lensed_images.shape[0], interval=60)

plt.close()
HTML(ani.to_jshtml())

Interestingly we see a lot more complexity in this example. The elliptical mass distribution allows for multiple images of the source to appear, rather than just a ring. If you download the tutorial for yourself, you can play around with the parameters to see all sorts of interesting configurations!

Note we made the source very small here to make the four images appear more dramatically.

## Point Mass Potential

For a point mass the lensing potential is:

$$\Psi(\vec{\theta}) = R_e\ln(|\vec{\theta}|)$$

Here we show the potential from a series of point masses with a gaussian distribution. For comparison we show the analytic potential field of an SIE. While these aren't exactly the same distribution, it is clear that they share similar general properties.

In [ ]:
F = 100
# Define a cosmology for the lensing
cosmology = caustics.FlatLambdaCDM()
# Define a point mass for the lens plane
sie_lens = caustics.SIE(
    cosmology=cosmology, x0=0.0, y0=0.0, q=0.99, phi=0, b=0.1, z_l=z_l
)
# Define a batched plane lens to combine many point masses
point_lens = caustics.BatchedPlane(
    cosmology=cosmology, lens=caustics.Point(cosmology=cosmology, z_l=z_l), z_l=z_l
)

thx, thy = caustics.utils.meshgrid(res, n_pix)
r = np.abs(np.random.randn(F)) * fov / 4
th = np.random.rand(F) * 2 * np.pi
point_locs = torch.tensor(
    np.stack((r * np.cos(th), r * np.sin(th)), axis=1), dtype=torch.float32
)
images = [torch.zeros(n_pix, n_pix)]
for n in range(1, F):
    images.append(
        point_lens.potential(
            thx,
            thy,
            z_s,
            [point_locs[:n, 0], point_locs[:n, 1], 0.6 * torch.ones(n)],
        )
    )

In [ ]:
# Create animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# Display the first frame of the image in the first subplot
im1 = ax1.imshow(np.zeros((n_pix, n_pix)), cmap="grey", interpolation="bilinear")
ax1.set_title("point masses [000]")
ax1.axis("off")
im2 = ax2.imshow(
    sie_lens.potential(thx, thy, z_s).numpy(),
    cmap="grey",
    interpolation="bilinear",
    extent=[-2, 2, -2, 2],
)
ax2.set_title("SIE")
ax2.axis("off")


def update(frame):
    """Update function for the animation."""
    # Update the image in the first subplot
    im1.set_array(images[frame].numpy())
    im1.set_clim(vmin=images[frame].min(), vmax=images[frame].max())
    ax1.set_title(f"point masses [{frame:03d}]")
    # im2.set_array(lensed_images[frame].numpy())

    return im1


ani = animation.FuncAnimation(fig, update, frames=F, interval=100)

plt.close()
HTML(ani.to_jshtml())

### Lens Equation Jacobian

The lens equation describes the motion of rays through a gravitational lensing system. It's jacobian encodes plenty of useful information about the system:

$$A_{ij}(\vec{\theta}) = \frac{\partial\beta_i(\vec{\theta})}{\partial\theta_j} = \delta_{ij} - \frac{\partial\alpha_i(\vec{\theta})}{\partial\theta_j} = \delta_{ij} - \frac{\partial\Psi(\vec{\theta})}{\partial\theta_i\partial\theta_j}$$

Where we have written out multiple ways to compute this jacobian based on whether you have access to the deflection field or the potential as the more computationally efficient quantity.

In [ ]:
cosmology = caustics.FlatLambdaCDM()
lens = caustics.SIE(cosmology=cosmology, x0=0.0, y0=0.0, q=0.6, phi=0, b=1.0, z_l=z_l)
theta_x, theta_y = caustics.utils.meshgrid(res, n_pix)
A = lens.jacobian_lens_equation(theta_x, theta_y, z_s)

In [ ]:
fig, axarr = plt.subplots(2, 2, figsize=(8, 8))
for i in range(2):
    for j in range(2):
        axarr[i, j].imshow(A[:, :, i, j].numpy(), origin="lower", cmap="seismic")
        axarr[i, j].axis("off")
        axarr[i, j].set_title(f"Jacobian component $A_{{{i}, {j}}}$")
plt.show()

### Magnification and Shear

The jacobian describes the local behavior of the lensing. We can break the jacobian into two parts and isotropic part and a antisymmetric part like so:

$$ A = A_{iso} + A_{anti} = \frac{1}{2}tr(A)\mathbb{I} + \left(A - \frac{1}{2}tr(A)\mathbb{I}\right)$$

Also for brevity we will use a shorthand for the components of $A$ based on the potential $\frac{\partial\Psi}{\partial\theta_i\partial\theta_j} = \Psi_{ij}$ so that now $A_{ij} = \delta_{ij}-\Psi_{ij}$. In this case the isotropoc component of the jacobian becomes:

$$A_{iso} = \left(1 - \frac{1}{2}(\Psi_{11} + \Psi_{22})\right)\mathbb{I} = (1-\kappa)\mathbb{I}$$

Which you can see is directly related to the convergence (see the [conversions](#relating-the-deflection-angle-convergence-and-potential) section). The remaining antisymmetric part of the jacobian looks like this:

$$A_{anti} = \left[\begin{matrix} 
-\frac{1}{2}(\Psi_{11} - \Psi_{22}) & -\Psi_{12} \\ 
-\Psi_{12} & \frac{1}{2}(\Psi_{11} - \Psi_{22})
\end{matrix}\right]$$

We define a pseudo vector $\vec{\gamma} = (\gamma_1, \gamma_2)$ where $\gamma_1 = \frac{1}{2}(\Psi_{11} - \Psi_{22})$ and $\gamma_2 = \Psi_{12}$ allowing us to write:

$$A_{anti} = \left[\begin{matrix} -\gamma_1 & -\gamma_2 \\ -\gamma_2 & \gamma_1 \end{matrix}\right] = -|\vec{\gamma}|\left[\begin{matrix} \cos(2\phi) & \sin(2\phi) \\ \sin(2\phi) & -\cos(2\phi) \end{matrix}\right]$$

The antisymmetric component, the shear is a $2\times2$ tensor (hence the $2\phi$) which represents the distortion of the source along a particular direction. 

In [ ]:
cosmology = caustics.FlatLambdaCDM()
lens = caustics.SIE(cosmology=cosmology, x0=0.0, y0=0.0, q=0.6, phi=0, b=1.0, z_l=z_l)
gamma_1, gamma_2 = lens.shear(theta_x, theta_y, z_s)

In [ ]:
fig, axarr = plt.subplots(1, 2, figsize=(8, 4))
axarr[0].imshow(gamma_1.numpy(), origin="lower", cmap="seismic")
axarr[0].axis("off")
axarr[0].set_title("Shear component $\\gamma_1$")
axarr[1].imshow(gamma_2.numpy(), origin="lower", cmap="seismic")
axarr[1].axis("off")
axarr[1].set_title("Shear component $\\gamma_2$")
plt.show()

Let see what happens when we apply only a shear to an image. this will create distortions along a particular axis.

In [ ]:
F = 100
hirez = 2
source = caustics.Pixelated(
    x0=0.0, y0=0.0, image=np.load("assets/Einstein.npy"), pixelscale=0.4 * res / hirez
)
cosmology = caustics.FlatLambdaCDM()
lens = caustics.ExternalShear(cosmology=cosmology, x0=0.0, y0=0.0, z_l=z_l)
sim = caustics.LensSource(
    lens, source, pixelscale=res / hirez, pixels_x=hirez * n_pix, z_s=z_s
)

gamma_1 = torch.linspace(-0.8, 0.8, F)
gamma_2 = torch.linspace(-0.8, 0.8, F)
lensed_images1 = torch.vmap(sim)(
    torch.stack((gamma_1, torch.zeros_like(gamma_1)), dim=1)
)
lensed_images2 = torch.vmap(sim)(
    torch.stack((torch.zeros_like(gamma_2), gamma_2), dim=1)
)

In [ ]:
# Create animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# Display the first frame of the image in the first subplot
im1 = ax1.imshow(lensed_images1[0].numpy(), cmap="grey")
ax1.set_title("Shear Field $\\gamma_1$")
ax1.axis("off")
im2 = ax2.imshow(
    lensed_images2[0].numpy(),
    cmap="grey",
)
ax2.set_title("Shear Field $\\gamma_2$")
ax2.axis("off")


def update(frame):
    """Update function for the animation."""
    # Update the image in the first subplot
    im1.set_array(lensed_images1[frame].numpy())
    ax1.set_title(f"Shear Field $\\gamma_1 = {gamma_1[frame].item():.2f}$")
    im2.set_array(lensed_images2[frame].numpy())
    ax2.set_title(f"Shear Field $\\gamma_2 = {gamma_2[frame].item():.2f}$")

    return im1, im2


ani = animation.FuncAnimation(fig, update, frames=F, interval=60, repeat=True)

plt.close()
HTML(ani.to_jshtml())

Finally, we can look at the magnification:

$$\mu = \frac{1}{det(A)} = \frac{1}{(1-\kappa)^2 - |\vec{\gamma}|^2}$$

Where we see that the magnification is a function of both the convergence and shear, though it can be readily computed from the jacobian matrix.

In [ ]:
cosmology = caustics.FlatLambdaCDM()
lens = caustics.SIE(cosmology=cosmology, x0=0.0, y0=0.0, q=0.6, phi=0, b=1.0, z_l=z_l)
mu = lens.magnification(theta_x, theta_y, z_s)

In [ ]:
plt.imshow(mu.log().numpy(), origin="lower", cmap="seismic", vmin=-6, vmax=6)
plt.colorbar()
plt.axis("off")
plt.title("Magnification $\log(\\mu)$")
plt.show()

The magnification is formally infinite along the [critical curve](#critical-lines-and-caustics) but since we are displaying a pixelated image we simply reach large values.

## Critical Lines and Caustics

Given the formula for the magnification $\mu = \frac{1}{det(A)}$ it should not be too surprising that interesting things happen when $det(A)=0$. In fact, the contour of positions where this happens is called the critical curve. It is possible to plot these critical curves for a given lens. These critical curves exist in the lens plane, if we raytrace them back to the source plane then they become caustics (ba dumb tis). Lets see what this looks like for a simple SIE.

In [ ]:
F = 50
cosmology = caustics.FlatLambdaCDM()
lens = caustics.SIE(cosmology=cosmology, x0=0.0, y0=0.0, phi=0, b=0.7, z_l=z_l)
q = torch.linspace(0.1, 1.0, F)
A = torch.stack(
    [lens.jacobian_lens_equation(theta_x, theta_y, z_s, q_.unsqueeze(0)) for q_ in q]
)
detA = torch.linalg.det(A)
images = torch.vmap(lambda q: lens.convergence(theta_x, theta_y, z_s, q.unsqueeze(0)))(
    q
)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4))


def update(frame):
    ax.clear()
    ax.imshow(
        images[frame].log(),
        origin="lower",
        label="convergence",
        extent=[-fov / 2, fov / 2, -fov / 2, fov / 2],
    )
    ax.plot([], [], color="#47378D", label="Convergence", linewidth=7)
    CS = ax.contour(
        theta_x, theta_y, detA[frame], levels=[0.0], colors="orange", linewidths=2
    )
    ax.plot([], [], color="orange", label="Critical curve", linewidth=2)
    paths = CS.allsegs[0]

    for path in paths:
        x1 = torch.tensor(list(float(vs[0]) for vs in path))
        x2 = torch.tensor(list(float(vs[1]) for vs in path))
        # raytrace the points to the source plane
        y1, y2 = lens.raytrace(x1, x2, z_s, q[frame].unsqueeze(0))

        # Plot the caustic
        ax.plot(y1, y2, color="r", label="Caustic", linewidth=2)
    ax.legend()
    ax.set_xlim([-2.5, 2.5])
    ax.set_ylim([-1.5, 1.5])
    ax.set_aspect("equal")
    ax.set_title(f"Critical Curve and Caustic (q = {q[frame].item():.2f})")


update(0)
ani = animation.FuncAnimation(fig, update, frames=F, interval=60, repeat=True)

plt.close()
HTML(ani.to_jshtml())

### Time Delay

On the cosmic scales of lensing, the finite speed of light becomes especially apparent. This means that when lensing causes multiple images of the same object to appear, they often actually represent the object at different times. There are in fact two reasons for this. One is that the light rays will have taken different paths to reach us the observer, and those different paths are different lengths. Since light travels at a finite speed, a longer path will have taken longer to get to us. The second reason is that gravitational fields bend spacetime and so a certain time delay is induced simply by being in a strong gravitational potential. It turns out that both these effects are often of a similar magnitude and so we need to account for them both. Luckily it is fairly straightforward to do so.

$$T = \frac{(1+z_l)D_sD_l}{cD_{ls}}\left(\frac{1}{2}|\vec{\hat{\alpha}}|^2 - \Psi\right)$$

The first term is the geometric time delay caused by different path lengths and is proportional to the physical deflection angle. The second term comes from the strength of the gravitational field called the *Shapiro Delay* and it is proportional to the lensing potential.


In [ ]:
F = 100
source = caustics.Sersic(x0=0.05, y0=0.1, q=1.0, phi=0.0, n=2, Re=0.05)
T = caustics.Param("T")  # time variable
source.Ie = (
    lambda p: torch.sin(5 * 2 * np.pi * (p["T"].value % (2 * np.pi))) ** 2
)  # periodic brightness
source.Ie.link(T)
cosmology = caustics.FlatLambdaCDM()
lens = caustics.SIE(cosmology=cosmology, x0=0.0, y0=0.0, q=0.5, phi=0, b=1.0, z_l=z_l)
t = torch.linspace(0, 1, F)
source_images = torch.vmap(lambda t: source.brightness(theta_x, theta_y, [t]))(t)
td_field = lens.time_delay(theta_x, theta_y, z_s) / 365
# td_field = td_field - td_field.min()
beta_x, beta_y = lens.raytrace(theta_x, theta_y, z_s)
lensed_images = torch.vmap(lambda t: source.brightness(beta_x, beta_y, [t - td_field]))(
    t
)

In [ ]:
plt.imshow(td_field.numpy(), origin="lower", cmap="grey")
plt.colorbar()
plt.axis("off")
plt.title("Time Delay Field")
plt.show()

In [ ]:
# Create animation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))

# Display the first frame of the image in the first subplot
im1 = ax1.imshow(
    source_images[0].numpy(),
    cmap="grey",
    interpolation="bilinear",
    vmin=0,
    vmax=source_images.max().item(),
)
ax1.set_title("unlensed")
ax1.axis("off")
im2 = ax2.imshow(
    lensed_images[0].numpy(),
    cmap="grey",
    interpolation="bilinear",
    vmin=0,
    vmax=2,
    extent=[-fov / 2, fov / 2, -fov / 2, fov / 2],
)
ax2.set_title("lensed")
ax2.axis("off")


def update(frame):
    """Update function for the animation."""
    # Update the image in the first subplot
    im1.set_array(source_images[frame].numpy())
    im2.set_array(lensed_images[frame].numpy())

    return im1, im2


ani = animation.FuncAnimation(fig, update, frames=lensed_images.shape[0], interval=60)

plt.close()
HTML(ani.to_jshtml())

## Multiplane Lensing

![Multiplane Lensing](assets/multiplanelensing.png)